### Функция ошибки (с регуляризацией)

Основная функция ошибки для одного элемента матрицы взаимодействий $ A_{ij} $:
$$
E_{ij} = (A_{ij} - U_i^T V_j)^2 + \lambda (||U_i||^2 + ||V_j||^2)
$$
Здесь:
- $ A_{ij} $ — фактическое взаимодействие (например, рейтинг).
- $ U_i $ и $ V_j $ — вектора скрытых факторов для пользователя $ i $ и товара $ j $, соответственно.
- $ U_i^T V_j $ — предсказанное взаимодействие.
- $ \lambda $ — коэффициент регуляризации для предотвращения переобучения.

### Вычисление градиента для каждого параметра

1. **Градиент по $ U_i $**

   Нам нужно минимизировать функцию ошибки $ E_{ij} $, изменяя $ U_i $. Для этого вычисляется частная производная ошибки по каждому элементу вектора $ U_i $:

   $$
   \frac{\partial E_{ij}}{\partial U_i} = -2 (A_{ij} - U_i^T V_j) V_j + 2 \lambda U_i
   $$
   - $ -2 (A_{ij} - U_i^T V_j) V_j $ — это градиент без регуляризации, который корректирует вектор $ U_i $ на основе ошибки предсказания.
   - $ 2 \lambda U_i $ — регуляризация, которая уменьшает значения элементов $ U_i $, чтобы избежать переобучения.

   Итоговое обновление $ U_i $ с использованием градиентного спуска:
   $$
   U_i \gets U_i + \alpha \left( (A_{ij} - U_i^T V_j) V_j - \lambda U_i \right)
   $$
   Здесь $ \alpha $ — скорость обучения (learning rate).

   **Аналогично, градиент по $ V_j $:**
   $$
   V_j \gets V_j + \alpha \left( (A_{ij} - U_i^T V_j) U_i - \lambda V_j \right)
   $$

In [138]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [133]:
# Загрузка и разделение данных
def load_and_split_data(file_path, test_size=0.3, random_state=42):
    df = pd.read_csv(file_path)
    train, test = train_test_split(df, test_size=test_size, random_state=random_state)
    return train, test

def create_cf_matrix(train, cf_type='user'):
    if cf_type == 'user':
        # Матрица пользователь-фильм
        return train.pivot(index='userId', columns='movieId', values='rating').fillna(0).to_numpy()
    elif cf_type == 'item':
        # Матрица фильм-пользователь
        return train.pivot(index='movieId', columns='userId', values='rating').fillna(0).to_numpy()
    else:
        raise ValueError("cf_type должен быть 'user' или 'item'")

In [145]:
# Загрузка данных
df = pd.read_csv('data/ml-latest-small/ratings.csv')
n_users = df['userId'].nunique()
n_items = df['movieId'].nunique()

# Создание индексов для пользователей и фильмов
user_ids = df['userId'].astype('category').cat.codes.values
item_ids = df['movieId'].astype('category').cat.codes.values

df['user_idx'] = user_ids
df['item_idx'] = item_ids

# Разделение данных на тренировочные и тестовые наборы
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Матрица рейтингов
def create_matrix(df, n_users, n_items):
    matrix = np.zeros((n_users, n_items))
    for row in df.itertuples():
        matrix[row.user_idx, row.item_idx] = row.rating
    return matrix

A_train = create_matrix(train_df, n_users, n_items)
A_test = create_matrix(test_df, n_users, n_items)
assert A_train.shape == A_test.shape

In [146]:
# Гиперпараметры
num_users, num_items = A_train.shape
latent_dim = 2  # Количество скрытых факторов
learning_rate = 0.01
reg_param = 0.1  # Параметр регуляризации
epochs = 1000

# Инициализация матриц скрытых факторов для пользователей и товаров
U = np.random.normal(scale=1./latent_dim, size=(num_users, latent_dim))
V = np.random.normal(scale=1./latent_dim, size=(num_items, latent_dim))

# Функция потерь (MSE + регуляризация)
def loss(A, U, V, reg_param):
    predicted = U.dot(V.T)
    mask = A > 0
    error = (A[mask] - predicted[mask])**2
    return np.sum(error) + reg_param * (np.sum(U**2) + np.sum(V**2))

# Градиентный спуск
with tqdm(total=epochs, desc="Epochs") as epoch_bar:
    for epoch in range(epochs):
        with tqdm(total=num_users*num_items, desc=f"Epoch {epoch+1}", leave=False) as step_bar:
            for i in range(num_users):
                for j in range(num_items):
                    if A_train[i, j] > 0:  # Обновляем только ненулевые элементы
                        # Вычисляем ошибку предсказания: e_ij = A_ij - U_i^T V_j
                        e_ij = A_train[i, j] - np.dot(U[i, :], V[j, :])

                        # Градиент для U_i: U_i = U_i + alpha * (e_ij * V_j - lambda * U_i)
                        U[i, :] += learning_rate * (e_ij * V[j, :] - reg_param * U[i, :])

                        # Градиент для V_j: V_j = V_j + alpha * (e_ij * U_i - lambda * V_j)
                        V[j, :] += learning_rate * (e_ij * U[i, :] - reg_param * V[j, :])
                    
                    # Обновляем прогресс-бар для каждого шага
                    step_bar.update(1)

            # Вычисляем ошибку после каждой эпохи
            current_loss = loss(A_test, U, V, reg_param)
            epoch_bar.set_postfix(Loss=current_loss)

        # Обновляем прогресс-бар для эпох
        epoch_bar.update(1)

Epochs:   1%|▏         | 14/1000 [01:58<2:19:32,  8.49s/it, Loss=4.82e+4]


KeyboardInterrupt: 

In [147]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Загрузка данных
df = pd.read_csv('data/ml-latest-small/ratings.csv')
n_users = df['userId'].nunique()
n_items = df['movieId'].nunique()

# Создание индексов для пользователей и фильмов
user_ids = df['userId'].astype('category').cat.codes.values
item_ids = df['movieId'].astype('category').cat.codes.values

df['user_idx'] = user_ids
df['item_idx'] = item_ids

# Разделение данных на тренировочные и тестовые наборы
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Матрица рейтингов
def create_matrix(df, n_users, n_items):
    matrix = np.zeros((n_users, n_items))
    for row in df.itertuples():
        matrix[row.user_idx, row.item_idx] = row.rating
    return matrix

R_train = create_matrix(train_df, n_users, n_items)
R_test = create_matrix(test_df, n_users, n_items)

# Реализация SVD с градиентным спуском
class SVD:
    def __init__(self, R, K=20, alpha=0.002, beta=0.02, iterations=100):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha  # Скорость обучения
        self.beta = beta  # Регуляризация
        self.iterations = iterations

    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R > 0])

        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        progress_bar = tqdm(total=self.iterations, desc="Training Progress")
        for iteration in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            progress_bar.set_postfix({'RMSE': f"{rmse:.4f}"})
            progress_bar.update(1)
        progress_bar.close()

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.predict(i, j)
            error = r - prediction

            # Обновление биасов
            self.b_u[i] += self.alpha * (error - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (error - self.beta * self.b_i[j])

            # Обновление скрытых факторов
            self.P[i, :] += self.alpha * (error * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (error * self.P[i, :] - self.beta * self.Q[j, :])

    def predict(self, i, j):
        return self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)

    def rmse(self):
        xs, ys = self.R.nonzero()
        predicted = []
        actual = []
        for x, y in zip(xs, ys):
            predicted.append(self.predict(x, y))
            actual.append(self.R[x, y])
        return np.sqrt(np.mean((np.array(predicted) - np.array(actual)) ** 2))

# Запуск модели
svd = SVD(R_train, K=2, iterations=15, alpha=0.001, beta=0.1)
svd.train()

# Функция для расчета RMSE на тестовом наборе
def calculate_rmse(R_test, svd):
    xs, ys = R_test.nonzero()
    predicted = []
    actual = []
    progress_bar = tqdm(total=len(xs), desc="Calculating RMSE")
    for x, y in zip(xs, ys):
        predicted.append(svd.predict(x, y))
        actual.append(R_test[x, y])
        progress_bar.update(1)
    progress_bar.close()
    return np.sqrt(np.mean((np.array(predicted) - np.array(actual)) ** 2))

rmse_test = calculate_rmse(R_test, svd)
print(f"Test RMSE: {rmse_test:.4f}")

Calculating RMSE: 100%|██████████| 30251/30251 [00:00<00:00, 189040.20it/s]

Test RMSE: 0.9227


In [143]:
A_train.shape, A_test.shape

((610, 8566), (610, 6124))